## LLM Chain（线性逻辑模型链）

`LLMChain`可能是查询LLM对象最受欢迎的方式之一。它使用提供的输入键值（如果可用，则还使用内存键值）格式化提示模板，将格式化字符串传递给LLM，然后返回LLM输出。下面我们展示了`LLMChain`类的附加功能。

In [1]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

## 其他运行LLM链的方法

除了所有`Chain`对象共享的`__call__`和`run`方法（有关详细信息，请参见[入门](../getting_started.ipynb)），LLMChain还提供了几种调用链逻辑的方法：

- `apply`允许您针对输入列表运行链：

In [2]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSocktastic!'},
 {'text': '\n\nTechCore Solutions.'},
 {'text': '\n\nFootwear Factory.'}]

- `generate`类似于`apply`，但是它返回一个`LLMResult`，而不是字符串。LLMResult通常包含诸如标记用法和完成原因等有用的生成。

In [3]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSocktastic!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'prompt_tokens': 36, 'total_tokens': 55, 'completion_tokens': 19}, 'model_name': 'text-davinci-003'})

- `predict`类似于`run`方法，但输入键被指定为关键字参数，而不是Python字典。

In [12]:
# Single input example
llm_chain.predict(product="colorful socks")

'\n\nSocktastic!'

In [14]:
# Multiple inputs example

template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

## 解析输出

默认情况下，即使基础的`prompt`对象具有输出解析器，`LLMChain`也不会解析输出。如果您想在LLM输出上应用该输出解析器，请使用`predict_and_parse`而不是`predict`，并使用`apply_and_parse`而不是`apply`。

使用`predict`：

In [24]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict()

'\n\nRed, orange, yellow, green, blue, indigo, violet'

使用`predict_and_parser`：

In [25]:
llm_chain.predict_and_parse()

['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

## 从字符串初始化

您还可以直接从字符串模板构造LLMChain。

In [16]:
template = """Tell me a {adjective} joke about {subject}."""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [18]:
llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'